In [1]:
# from qiskit.opflow import I, Z, X, Y, SparsePauliOp
import numpy as np
# from juices import Juices
# 假设有 3 台机器和 3 个操作
m = 3 # 机器数量
N = 2 # 表示机器的二进制 (0, 1)
n = 2 # 操作数量
l = 1  # 用于表示 C_max 的二进制位数

# 处理时间矩阵 (例如)
p = np.array([[1, 2, 3],[3, 1, 2]])

# 非法数组(用于表示机器数)(1, 1)
b = np.array([1, 1])

# 常数
K1 = 10
P = 5

# 量子比特数量
num_qubits = n * N + l

In [3]:
def append__zz_term( qc, qubit1, qubit2, gamma):
    qc.cx(qubit1, qubit2)
    qc.rz(gamma, qubit2)
    qc.cx(qubit1, qubit2)

def append__z_term(qc, qubit, gamma):
    qc.rz(gamma, qubit)
def append__mixer_term(qc, qubit, beta):
    qc.rx(2 * beta, qubit)

In [2]:
def get_cost_circuit1(gamma, n, m, l, K1, P, p, qc):
    # beta, gamma = params[:len(params)//2], params[len(params)//2:]
    for h in range(l):
        # print(m * n + h)
        coef = - 2**h
        append__z_term(qc, m * n + h, coef * gamma)
    return qc

In [ ]:
def get_cost_circuit2(gamma, n, m, l, K1, P, p, qc):
    # beta, gamma = params[:len(params)//2], params[len(params)//2:]
    for i in range(n):
        for j in range(m):
            append__z_term(qc, i * m + j, (2 - m) * K1*gamma)
    # for i in range(n):
        for j1 in range(m):
            for j2 in range(m):
                if j1 != j2:
                    append__zz_term(qc, i * m + j1, i * m + j2, K1*gamma)
    return qc

In [ ]:
def get_cost_circuit3(gamma, n, m, l, K1, P, p, qc):
    # beta, gamma = params[:len(params)//2], params[len(params)//2:]
    for j in range(m):
        for i in range(n):
            coef = -1/2 * (np.sum(p[:, j]) - ( 2 ** l-1)) * P
            append__z_term(qc, i * m + j, 2 * coef * p[i,j] * gamma)
        for h in range(l):
            coef = -1/2 * (np.sum(p[:, j]) - ( 2 ** l-1)) * P
            append__z_term(qc, m * n + h, -2 * (2 ** h) * coef * gamma)
        for i1 in range(n):
            for i2 in range(n):
                if i1 != i2:
                    coef = p[i1, j] * p[i2, j] * P
                    append__zz_term(qc, i1 * m + j, i2 * m + j, coef * gamma)
        for i in range(n):
            for h in range(l):
                coef = -1 * p[i,j] * 2 ** h * P
                append__zz_term(qc, i * m + j, m * n + h, coef * gamma)
        for h1 in range(l):
            for h2 in range(l):
                if h1 != h2:
                    coef = 2 ** h1 * 2 ** h2 * P
                    append__zz_term(qc, m * n + h1, m * n + h2, coef * gamma)
    return qc